# Capstone Project - The Battle of the Neighborhoods (Week 1)
### Applied Data Science Capstone by IBM/Coursera

## Data <a name="data"></a>

To assess a business case, the well-known SWOT business analysis methodology tell us to look into these four aspects, namely, Strengths, Weakness, Opportunities and Threats. As strengths and weakness are internal to the business itself, here we concentrate on the opportunities and threats aspect. 

High restaurant density can represent threats as it means more competitions. On the other hand, it can also represent opportunities. For example, a cluster of French restaurants may have some kind of “branding” effect as this may foster a reputation of being a go-to place for French foods.

The other type of opportunities/threats are represented by what type of the potential customer base the restaurant location has - Is it in a high traffic area, i.e. busy and bustling with people, and whether the people ready to spend money in restaurant?
Therefore, to prediction model will need to take into account of the above ‘features’ of a location in order to predict whether it is a good or bad location. 

For this purpose, firstly, the datasets I am going to use to represent restaurant density are:
* For a given location, how many nearby *Chinese restaurants* and how closed are they?
* For a given location, how many nearby *similar types of restaurants* and how closed are they? For similar restaurants, I refer to Asian type such as Japanese, Korean, Thai, Indian, Malay, etc.
* For a given location, how many nearby *non-similar types of restaurants* and the proximity of them?

The above datasets can be obtained by using the **Foursquare API**.

Secondly, for the data about the potential customer base, there is no readily available source. However, I found two data sources I believe to be good representation of potential customer base:
1. ATM (i.e. bank automated teller machine) location data. ATMs are placed by bank in places where there are high traffic of people who want to withdraw cash. Therefore, the proximity of ATMs can give good indication for the traffic volume and whether people are spending money. The source of ATM I use for this project is **Overpass API (OpenStreetMap)**.
2. Public Transportation link data. Underground and bus are two major public transportation links within London. **London Underground passenger counts** published by the London Transportation Department can be used as a good indication of traffic volume. 

Finally, we also need the historical data of the prediction target (i.e. how good/bad a location for a Chinese restaurant) for train and verify the prediction model. Ideally, past profitability of the existing Chinese restaurants would be the ideal historical data but such data are not readily available. Another good representative data would be the venue statistics from Foursquare, unfortunately, I found that for many Chinese restaurants in London, Foursquare does not provide such statistics. Therefore, for the purpose of this project, I use the “number of likes” in the Foursquare venues information.
The recap, the data I am going to use to train and verify the prediction model are:
* Data for the prediction target: “number of likes” in the **Foursquare venues information**; 
* Data for the features: 
  * number of nearby Chinese restaurants and average distance (**Foursquare API**)
  * number of nearby similar types of restaurants and average distance (**Foursquare API**)
  * number of nearby non-similar types of restaurants and average distance (**Foursquare API**)
  * number of nearby ATMs locations and average distance (**Overpass API -OpenStreetMap**) 
  
  * **London Underground passenger counts**.
